# Quelques tests sur les tenseurs


Bon, que se passe-t-il si on fait une multiplication matricielle avec des tenseurs de differentes tailles.

Déja, on fait les install et les imports

In [ ]:
!pip install torchviz

In [ ]:
import numpy as np
import torch
from torchviz import make_dot

import einops

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Multiplication de vecteurs

Commençons par deux vecteurs (des tenseurs d'ordre 1).
Le produit matriciel de deux vecteurs est (dans torch), un produit scalaire

Une premiere remarque :
**Un tenseur vectoriel n'a pas de direction,
Il n'y a pas de vecteur ligne / vecteur colonne. Il n'y a que des tenseurs d'ordre 1**


### Preparation des données :

- 2 vecteurs : $x1$ et $x2$ de taille 3
- 1 vecteur : $xt1$ de taille 2 (servira plus loin)

In [ ]:
x1 = np.array([1,2,3])
x2 = np.array([2,3,4])

xt1 = np.array([1,1])

x1_tensor = torch.from_numpy(x1).float().to(device)
x2_tensor = torch.from_numpy(x2).float().to(device)
xt1_tensor = torch.from_numpy(xt1).float().to(device)

print(x1)

print("x1",x1_tensor)
print("x2",x2_tensor)
print("xt1",xt1_tensor)

x1_tensor


[1 2 3]
x1 tensor([1., 2., 3.], device='cuda:0')
x2 tensor([2., 3., 4.], device='cuda:0')
xt1 tensor([1., 1.], device='cuda:0')


tensor([1., 2., 3.], device='cuda:0')

### calcul du produit

In [ ]:
# Produit de deux vecteurs : produit scalaire
prod = x1_tensor @ x2_tensor
print(prod)

# on peut récuperer un float
res = prod.item()
print(res, type(res))

print("devrait etre 20")

tensor(20., device='cuda:0')
20.0 <class 'float'>
devrait etre 20


pour comparaison, si on considérait des matrices, il faudrait :

une matrice ligne x une matrice colonne

comme suit

In [ ]:
# unsqueeze

print(x1_tensor.unsqueeze(0))

# on unsqueeze la derniere dimension
print(x2_tensor.unsqueeze(-1))


tensor([[1., 2., 3.]], device='cuda:0')
tensor([[2.],
        [3.],
        [4.]], device='cuda:0')


In [ ]:
# multiplication vecteur vecteur sous forme de deux matrices
prod = x1_tensor.unsqueeze(0) @ x2_tensor.unsqueeze(-1)
print(prod)

# on peut récuperer un float
res = prod[0][0].item()
print(res, type(res))

print("devrait etre 20")

tensor([[20.]], device='cuda:0')
20.0 <class 'float'>
devrait etre 20


## Multiplication matrice vecteur

Un tenseur vectoriel n'a pas de direction,
Il n'y a pas de vecteur ligne / vecteur colonne. Il n'y a que des tenseurs d'ordre 1

il faut juste que le nombre de colonnes de la matrice soit la taille du vecteur.

A part ca, pas de surprises.


In [ ]:
M = np.array([[1,2,3],[1,1,1]])


M_tensor = torch.from_numpy(M).float().to(device)
print (M_tensor)

tensor([[1., 2., 3.],
        [1., 1., 1.]], device='cuda:0')


In [ ]:
# multiplication Matrice vecteur

print(M_tensor,"\n MULTIPLIE PAR \n",x2_tensor)

prod = M_tensor @ x2_tensor
print("\n resu\n",prod)

print("========================")
print(xt1_tensor,"\n MULTIPLIE PAR \n",M_tensor)

prod = xt1_tensor @ M_tensor
print("\n resu\n",prod)



tensor([[1., 2., 3.],
        [1., 1., 1.]], device='cuda:0') 
 MULTIPLIE PAR 
 tensor([2., 3., 4.], device='cuda:0')

 resu
 tensor([20.,  9.], device='cuda:0')
tensor([1., 1.], device='cuda:0') 
 MULTIPLIE PAR 
 tensor([[1., 2., 3.],
        [1., 1., 1.]], device='cuda:0')

 resu
 tensor([2., 3., 4.], device='cuda:0')


## Multiplication matrice matrice

configuration typique d'un réseau de neurone traditionnel (dans ma tête)

- On a un vecteur par exemple, de shape $(n,)$
- on batche les vecteurs par paquets de taille $s_{batch}$
- ca nous fait une matrice $X$ de shape $(n,s_{batch})$ (**chaque X est une colonne**)
- on a une matrice de poids de shape $(n_{out},n)$
- on fait le calcul suivant : $out = X \times X + b$


Ca, c'est dans ma tête ou sur mes dessins, parce que je fais toujours des multiplications matricielles à droite.


### Remarque importante

**EN VRAI : On batche sur la premiere dimension**

- On a un vecteur par exemple, de shape $(n,)$
- on batche les vecteurs pour en faire une matrice $X$ de shape $(s_{batch},n)$ (chaque $x$ est **une ligne**)
- on a une matrice de poids de shape $(n_{out},n)$
- on fait le calcul suivant : $out = X \times W^\intercal + b$

$out$ est de shape $(s_{batch},n)$

### Revenons à nos multiplications

Ici, pas de surprise.

Pour les tests, on va concatener les 2 vecteurs $x1$, $x2$ pour faire une matrice 3x2 $M1$ (comme dans ma tête)

On calculera alors M x M1

- comme on veut créer une nouvelle dimension (on passe de dim 1 à dim 2),
on utilise *torch.stack*, pas *torch.cat*
- comme on veut rester coherent dans le $ M \times M1$, on les stack pour que les x initiaux soient "en colonne". La nouvelle direction est donc 1.

J'ai encore batché dans la derniere dimension...


In [ ]:
M1 = torch.stack((x1_tensor, x2_tensor),dim=1)

print(M1)


tensor([[1., 2.],
        [2., 3.],
        [3., 4.]], device='cuda:0')


In [ ]:
prod = M_tensor @ M1

print(M_tensor,"\n MULTIPLIE PAR \n",M1)

prod = M_tensor @ M1
print("\n resu\n",prod)


tensor([[1., 2., 3.],
        [1., 1., 1.]], device='cuda:0') 
 MULTIPLIE PAR 
 tensor([[1., 2.],
        [2., 3.],
        [3., 4.]], device='cuda:0')

 resu
 tensor([[14., 20.],
        [ 6.,  9.]], device='cuda:0')


### Matrice x tenseur d'ordre 3

Ici, on commence ce qui m'intéresse, car c'est ce qui se passe quand on passe
une phrase dans un transformer.

- chaque mot (token) est encodé en un vecteur de shape $(s_{token},)$
- Une phrase est une sequence de token. c'est une matrice de shape $(s_{seq},s_{token})$
- un batch de phrase est un tenseur de shape $(s_{batch},s_{seq},s_{token})$

Prenons un MLP qui travaillerait sur chaque token indépendamment,
pour chaque composante du token, il calcule une nouvelle sortie.

Sa matrice est donc de shape $(s_{token},s_{token})$

- Si on applique cette matrice à un token (vecteur), la sortie est un vecteur, tout se passe comme prévu.
- Si on applique cette matrice à une sequence de token (une matrice), la sortie est le resultat d'une multiplication matricielle. Il faut **faire attention à ne pas faire d'opérations entre composantes de tokens différents**.


Comme le montrent les exemples ci dessous, il faut faire $input \times Matrice$
plutot que $Matrice \times input$ pour avoir le comportement attendu :

quand on ajoute une nouvelle dimension, on répéte simplement l'opération de la matrice sur cette nouvelle dimension.


In [ ]:

# prenons une matrice qui calcule, pour un token, la somme et la différence de ses composantes
W = np.array([[1,1],[1,-1]])


W = torch.from_numpy(W).float().to(device)
print ("W",W)

# le premier token de la premiere phrase est [1,2]. le second est [3,4]
# le premier token de la seconde phrase est [5,6]. le second est [7,8]
X = np.array([[[1,2],[3,4]],[[5,6],[7,8]]])
X = torch.from_numpy(X).float().to(device)
print("\nX.shape",X.shape)
print ("\nX\n",X)


print ("\n==========MULTIPLICATIONS PAR LA DROITE ===============\n")
## Ceci fait ce que je pense : W x [1,2] -> (3, -1)
print("\n X[0][0]\n",X[0][0])
res = W @ X[0][0]
print("\nres\n",res)

## Voyons ceci  : W x [[1,2],[3,4]]
# visiblement, ca fait la somme sur les colonnes, et la différence des colonnes. [4,6][-2,-2])
print("\n X[0]\n",X[0])
res = W @ X[0]
print("\nres\n",res)

## Voyons ceci  : W x [[1,2],[3,4]],[[5,6],[7,8]]
# visiblement, ca fait comme avant, mais plusieurs fois
print("\n X\n",X)
res = W @ X
print("\nres\n",res)

print ("\n==========MULTIPLICATIONS PAR LA GAUCHE ===============\n")
## Ceci fait ce que je pense : [1,2] x [1, 1],[1,-1]  -> (3, -1)
print("\n X[0][0]\n",X[0][0])
res = X[0][0] @ W
print("\nres\n",res)

## ceci fait bien la somme et la différence des composantes de chaque token
## d'une phrase
print("\n X[0]\n",X[0])
res = X[0] @ W
print("\nres\n",res)

## ceci fait bien la somme et la différence des composantes de chaque token
# de chaque phrase
print("\n X\n",X)
res = X @ W
print("\nres\n",res)



W tensor([[ 1.,  1.],
        [ 1., -1.]], device='cuda:0')

X.shape torch.Size([2, 2, 2])

X
 tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]], device='cuda:0')

==========MULTIPLICATIONS PAR LA DROITE ===============


 X[0][0]
 tensor([1., 2.], device='cuda:0')

res
 tensor([ 3., -1.], device='cuda:0')

 X[0]
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0')

res
 tensor([[ 4.,  6.],
        [-2., -2.]], device='cuda:0')

 X
 tensor([[[1., 2.],
         [3., 4.]],

        [[5., 6.],
         [7., 8.]]], device='cuda:0')

res
 tensor([[[ 4.,  6.],
         [-2., -2.]],

        [[12., 14.],
         [-2., -2.]]], device='cuda:0')

==========MULTIPLICATIONS PAR LA GAUCHE ===============


 X[0][0]
 tensor([1., 2.], device='cuda:0')

res
 tensor([ 3., -1.], device='cuda:0')

 X[0]
 tensor([[1., 2.],
        [3., 4.]], device='cuda:0')

res
 tensor([[ 3., -1.],
        [ 7., -1.]], device='cuda:0')

 X
 tensor([[[1., 2.],
         [3., 4.]],

        

In [ ]:
print ("\n==========BATCHS (PAR LA GAUCHE) ===============\n")
batch = torch.stack((X,2*X,3*X))
print("\nun batch de 3 phrases\n",batch)

res = batch @ W
print("resultat",res)


==========BATCHS (PAR LA GAUCHE) ===============


un batch de 3 phrases
 tensor([[[[ 1.,  2.],
          [ 3.,  4.]],

         [[ 5.,  6.],
          [ 7.,  8.]]],


        [[[ 2.,  4.],
          [ 6.,  8.]],

         [[10., 12.],
          [14., 16.]]],


        [[[ 3.,  6.],
          [ 9., 12.]],

         [[15., 18.],
          [21., 24.]]]], device='cuda:0')
resultat tensor([[[[ 3., -1.],
          [ 7., -1.]],

         [[11., -1.],
          [15., -1.]]],


        [[[ 6., -2.],
          [14., -2.]],

         [[22., -2.],
          [30., -2.]]],


        [[[ 9., -3.],
          [21., -3.]],

         [[33., -3.],
          [45., -3.]]]], device='cuda:0')


## INTERPRETATIONS DE CES MULTIPLICATIONS : BROADCASTING

lu quelque part : "The matrix multiplication(s) are done between the last two dimensions. The remaining first three dimensions are broadcast and are ‘batch’"

testons ca. Ici, on a

- un tenseur $W_{batch}$ de shape $[2,2,2]$ qui représente 2 matrices empilées
- un tenseur $batch$ de shape $[3,2,2,2]$ qui représente nos inputs. Cela pourrait être 3 images à deux canaux.

Pour bien s'assurer de ce que l'on fait, vu les égalités de longueur des shape, précisons l'ordre des canaux pour $batch$ : $[B, C, H, W]$

Pour la matrice, la shape est $[C, H, W]$

On calcule $ batch \times W_{batch}$.

Le résultat est surprenant :

- la premiere matrice de $W_{batch}$ multiplie le premier canal d'une image.
- la seconde matrice de $W_{batch}$ multiplie le second canal d'une image.
- ces opérations sont répétées pour chaque image



Ceci est lié au fait que la multiplication **broadcaste** les données : https://docs.pytorch.org/docs/stable/notes/broadcasting.html#broadcasting-semantics

le **broadcast** dans la multiplication matricielle a des comportements differents en fonction des tailles des dimensions respectives des tenseurs.

Pour savoir s'il faut et si on peut broadcaster, **on regarde les dimensions en partant de la fin et en remontant vers le début**

1. Pour *matmult* : les deux dernieres dimensions doivent être compatibles pour une multiplication matricielle.

2. comme la troisieme dimension en partant de la fin est de même taille dans les deux tenseurs, tout se passe comme si on appliquait les calculs dans les dimensions suivantes de façon parallele. Ceci explique comment on ferait un calcul d'un tenseur de shape $[C,H,W] \times$ un tenseur de shape $[C,H,W]$ :
**chaque matrice $[H,W]$ des inputs est multipliée par la matrice $[H,W]$ correspondante du tenseur.**

3. Pour la partie batch, la 4eme dimension en partant de la fin n'existe pas dans W. Pas de problème, le broadcast va la créer et dupliquer les données.
Pour imaginer ca, disons qu'un scalaire $a$, ca peut se broadcaster en un vecteur $[a,a,a,a]$. **Notons que** $a$ **pourrait être un vecteur ou une matrice, ca marcherait pareil**. Donc le tenseur $W_{batch}$ va être étendu pour
appliquer l'opération 2 pour chaque "phrase" de $batch$

4. on s'en servira plus loin, mais une dimension de 1 peut aussi se broadcaster par duplication. Pour imaginer ca, disons qu'une matrice 1,2 telle que $[[1,2,3]]$ peut se broadcaster en une matrice 3,2 : $[[1,2,3],[1,2,3],[1,2,3]]$


In [ ]:
# on garde le X précédent, on prépare une liste de matrice.
# W1 va conserver la premiere composante d'un token.
W1 = torch.from_numpy(np.array([[1,0],[0,0]])).float().to(device)

W_batch = torch.stack((W,W1))
print("\nW_batch\n",W_batch)

res = batch @ W_batch

print("\nX\n",batch)

print("\nres\n",res)

print("batch.shape",batch.shape,"W.shape",W_batch.shape,"res.shape :",res.shape)


W_batch
 tensor([[[ 1.,  1.],
         [ 1., -1.]],

        [[ 1.,  0.],
         [ 0.,  0.]]], device='cuda:0')

X
 tensor([[[[ 1.,  2.],
          [ 3.,  4.]],

         [[ 5.,  6.],
          [ 7.,  8.]]],


        [[[ 2.,  4.],
          [ 6.,  8.]],

         [[10., 12.],
          [14., 16.]]],


        [[[ 3.,  6.],
          [ 9., 12.]],

         [[15., 18.],
          [21., 24.]]]], device='cuda:0')

res
 tensor([[[[ 3., -1.],
          [ 7., -1.]],

         [[ 5.,  0.],
          [ 7.,  0.]]],


        [[[ 6., -2.],
          [14., -2.]],

         [[10.,  0.],
          [14.,  0.]]],


        [[[ 9., -3.],
          [21., -3.]],

         [[15.,  0.],
          [21.,  0.]]]], device='cuda:0')
batch.shape torch.Size([3, 2, 2, 2]) W.shape torch.Size([2, 2, 2]) res.shape : torch.Size([3, 2, 2, 2])


### Broadcasting reloaded

Vu la doc, on va tester ceci : appliquer une deuxieme transformation à chacun de nos tokens.

On va modifier les inputs pour creer une dimension juste avant les deux dernieres, pour avoir : $[B, C,1, H, W]$

la matrice est de shape : [2,H,W]

On calcule X @ W

cette fois ci, chaque matrice H,W des données passe dans chacune des deux matrices de traitement.

Le broadcasting a en fait dupliqué les données des dimensions suivantes dans la dimension de taille 1 pour égaler la taille 2. Puis on applique la strat précédente.

**Ce sont des manipulations comme celle ci (et celles d'avant) qui permettent de faire de l'attention spatiale ou temporelle à moindre frais** : on va selectionner des vecteurs pertinents par permutation, pour les batcher.

**a noter : dans un cadre opérationnel, vu les résultats ci dessous, il faudrait peut être permuter les dimensions du résultat** pour que les 2 calculs effectués sur chaque token soient la deuxieme dimension en partant de la fin.

En l'état, à la sortie, j'ai une shape $[B,nb_{op},seq, embed]$


In [ ]:
X_reshaped = batch.reshape(3,2,1,2,2)
#print ("\nbatch reshaped\n",batch_reshaped)

res = X_reshaped @ W_batch
print("\nres\n",res)

print("X.shape",X_reshaped.shape,"W.shape",W_batch.shape,"res.shape :",res.shape)


res
 tensor([[[[[ 3., -1.],
           [ 7., -1.]],

          [[ 1.,  0.],
           [ 3.,  0.]]],


         [[[11., -1.],
           [15., -1.]],

          [[ 5.,  0.],
           [ 7.,  0.]]]],



        [[[[ 6., -2.],
           [14., -2.]],

          [[ 2.,  0.],
           [ 6.,  0.]]],


         [[[22., -2.],
           [30., -2.]],

          [[10.,  0.],
           [14.,  0.]]]],



        [[[[ 9., -3.],
           [21., -3.]],

          [[ 3.,  0.],
           [ 9.,  0.]]],


         [[[33., -3.],
           [45., -3.]],

          [[15.,  0.],
           [21.,  0.]]]]], device='cuda:0')
X.shape torch.Size([3, 2, 1, 2, 2]) W.shape torch.Size([2, 2, 2]) res.shape : torch.Size([3, 2, 2, 2, 2])


## Tenseurs, reshape et flatten


In [ ]:
imR = np.array([[1,2,3,4],[5,6,7,8]])
imG = np.array([[11,12,13,14],[15,16,17,18]])

im = np.stack([imR,imG])

im = torch.from_numpy(im).float().to(device)
print(im)

C = 2
H = 2
W = 4

# On reshape en patch de taille 1x1...
im_reshaped = im.reshape(2,H*W)
print("\n après reshape\n")
print(im_reshaped)

print("ci dessus, j'ai tous les pixels d'un canal en dimension finale.")

# Je permute pour avoir chaque patch en dimension finale.
im_permute = im_reshaped.permute(1,0)
print(im_permute)

tensor([[[ 1.,  2.,  3.,  4.],
         [ 5.,  6.,  7.,  8.]],

        [[11., 12., 13., 14.],
         [15., 16., 17., 18.]]], device='cuda:0')

 après reshape

tensor([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.],
        [11., 12., 13., 14., 15., 16., 17., 18.]], device='cuda:0')
ci dessus, j'ai tous les pixels d'un canal en dimension finale.
tensor([[ 1., 11.],
        [ 2., 12.],
        [ 3., 13.],
        [ 4., 14.],
        [ 5., 15.],
        [ 6., 16.],
        [ 7., 17.],
        [ 8., 18.]], device='cuda:0')


### Idem mais avec des patchs

In [ ]:


imR = np.array([[1,2,3,4],[5,6,7,8],[9,10,11,12],[13,14,15,16],[17,18,19,20],[21,22,23,24]])
imG = np.array([[101,102,103,104],[105,106,107,108],[109,110,111,112],[113,114,115,116],[117,118,119,120],[121,122,123,124]])

im = np.stack([imR,imG])

im = torch.from_numpy(im).float().to(device)
print(im)

C = 2
H = 6
W = 4

# On reshape en patch de taille 1x2...

sp_i = 3 # taille des patchs en nb lignes
sp_j = 2 #taille des patchs en nb colonnes

im_reshaped = im.reshape(C,H//sp_i,sp_i, W//sp_j,sp_j)
print("\n après reshape\n")
print(im_reshaped)

print ("\nregardons juste dans le canal rouge im_reshaped[0]\n")
print(im_reshaped[0])
print ("\nregardons le second patch de ligne des ligne du canal rouge im_reshaped[0][1]")
print(im_reshaped[0][1])


print("ci dessus, j'ai bien tout ce qui correspond au deuxieme patch de ligne, découpé en patchs de colonnes")
print("le problème est que la dimension suivante est du numéro de ligne dans le patch.")
print("par exemple, 3eme ligne du deuxieme patch de ligne : im_reshaped[0][1][2]\n")
print(im_reshaped[0][1][2])


print("-------------------------")

# Je permute pour avoir chaque patch en dimension finale.
im_permute = im_reshaped.permute(1,3,0,2,4)
print(im_permute)

print("ci dessus, c'est pas mal, j'ai reuni mes données de chaque patch sur les 3 dernieres dimensions\n")
print("reste plus qu'a flatten le tout proprement")

# Je flatten les dimensions finales
im_flattened_patches = im_permute.flatten(2,4)
print(im_flattened_patches)
print("\nci dessus, j'ai une matrice 2x2 de vecteurs\n")

# Je flatten les dimensions 0 et 1 correspondant aux numéros de patchs en i et j
im_flattened = im_flattened_patches.flatten(0,1)

print(im_flattened)

print("\nci dessus, j'ai bien une liste de 4 patchs de vecteurs\n")

print("\n shape des données : (4, 2x3x2) : ", im_flattened.shape)

tensor([[[  1.,   2.,   3.,   4.],
         [  5.,   6.,   7.,   8.],
         [  9.,  10.,  11.,  12.],
         [ 13.,  14.,  15.,  16.],
         [ 17.,  18.,  19.,  20.],
         [ 21.,  22.,  23.,  24.]],

        [[101., 102., 103., 104.],
         [105., 106., 107., 108.],
         [109., 110., 111., 112.],
         [113., 114., 115., 116.],
         [117., 118., 119., 120.],
         [121., 122., 123., 124.]]], device='cuda:0')

 après reshape

tensor([[[[[  1.,   2.],
           [  3.,   4.]],

          [[  5.,   6.],
           [  7.,   8.]],

          [[  9.,  10.],
           [ 11.,  12.]]],


         [[[ 13.,  14.],
           [ 15.,  16.]],

          [[ 17.,  18.],
           [ 19.,  20.]],

          [[ 21.,  22.],
           [ 23.,  24.]]]],



        [[[[101., 102.],
           [103., 104.]],

          [[105., 106.],
           [107., 108.]],

          [[109., 110.],
           [111., 112.]]],


         [[[113., 114.],
           [115., 116.]],

          [[11


### on code des video

Bon. on va se faire une video de 2 images.
chaque image a :
- 2 canaux
- une taille de 6x4

la seconde image est juste l'opposé de la premiere.

nos données ont une shape : (frames,C,H,W)

Eventuellement, on pourra faire un batch de video en stackant $(vid, 2.vid)$

on fait des patchs de taille $[tp,Hp,Wp] = [2,3,2]$

ca fait 2x2 patchs

In [ ]:
# une fonction pour faire les patchs
def extract_patches(video,C = 2,H = 6,W = 4,frames=2, sp_i=3,sp_j=2):
  im_reshaped = video.reshape(frames,C,H//sp_i,sp_i, W//sp_j,sp_j)

  # Je permute pour avoir chaque patch en dimension finale.
  # shape apres : [ni_patch, nj_patch, frames,C,Hp,Wp]
  im_permute = im_reshaped.permute(2,4,0,1,3,5)
  #print(im_permute)

  # Je flatten les dimensions finales
  #im_permute = im_permute.flatten(2,5)
  #print(im_permute)
  #print("\nci dessus, j'ai une matrice 2x2 de vecteurs\n")

  # Je flatten les dimensions 0 et 1 correspondant aux numéros de patchs en i et j
  # shape après : [ni_patch * nj_patch, frames,C,Hp,Wp]
  im_flattened = im_permute.flatten(0,1)

  return im_flattened


In [ ]:
vid = torch.stack((im,-im))
batch_vid = torch.stack((vid,2*vid))

print(vid.shape)
print ("\nvid\n", vid)


patches = extract_patches(vid)
print("\npatches.shape\n",patches.shape)
print("\n patches\n",patches)

print("\n=====preparation pour travail spatiotemporel=====\n")
# shape avant : [ni_patch * nj_patch, frames,C,Hp,Wp]
p_spatiotemp = patches.permute(1,0,2,3,4).reshape(2*4,-1)
print(p_spatiotemp)
print("\np_spatiotemp.shape\n",p_spatiotemp.shape)
print("PARFAIT")


print("\n=====preparation pour travail spatial=====\n")
# shape avant : [ni_patch * nj_patch, frames,C,Hp,Wp]
p_spatial = patches.permute(1,0,2,3,4).reshape(2,4,-1)
print(p_spatial)
print("\np_spatial.shape\n",p_spatial.shape)
print("PARFAIT")

print("\n =====preparation pour travail temporel====\n")
# shape avant : [ni_patch * nj_patch, frames,C,Hp,Wp]
p_temp = einops.rearrange(patches,"np t c h w -> np h w (t c) ")
print(p_temp)
print("\np_temp.shape\n",p_temp.shape)
print("je ne sais pas si c'est PARFAIT")

torch.Size([2, 2, 6, 4])

vid
 tensor([[[[   1.,    2.,    3.,    4.],
          [   5.,    6.,    7.,    8.],
          [   9.,   10.,   11.,   12.],
          [  13.,   14.,   15.,   16.],
          [  17.,   18.,   19.,   20.],
          [  21.,   22.,   23.,   24.]],

         [[ 101.,  102.,  103.,  104.],
          [ 105.,  106.,  107.,  108.],
          [ 109.,  110.,  111.,  112.],
          [ 113.,  114.,  115.,  116.],
          [ 117.,  118.,  119.,  120.],
          [ 121.,  122.,  123.,  124.]]],


        [[[  -1.,   -2.,   -3.,   -4.],
          [  -5.,   -6.,   -7.,   -8.],
          [  -9.,  -10.,  -11.,  -12.],
          [ -13.,  -14.,  -15.,  -16.],
          [ -17.,  -18.,  -19.,  -20.],
          [ -21.,  -22.,  -23.,  -24.]],

         [[-101., -102., -103., -104.],
          [-105., -106., -107., -108.],
          [-109., -110., -111., -112.],
          [-113., -114., -115., -116.],
          [-117., -118., -119., -120.],
          [-121., -122., -123., -124.]]]]